## 搜狗实验室新闻数据文本分类深度学习案例
### ———Tensorflow+CNN深度学习全流程

### 一、项目简介
tensorflow是谷歌开源的深度学习框架，是进行深度学习的坚船利炮。此文基于“搜狗实验数据库”的海量新闻数据，全流程展示如何基于tensorflow采用CNN算法实现文章的分类。方便学习者全面地理解深度学习及NLP文本分析的原理和实现步骤。

### 二、数据预处理
* 机器学习中，最为基础也是最为耗时的一项工作就是数据预处理。如何将海量数据进行预处理，进而得到数据处理和机器学习阶段所需要的有效素材是一项非常重要的工作。
* 此文将以海量新闻xml原始数据处理为例，展示如何有效第进行数据预处理工作。

#### 1、了解原始数据特征
* 该数据来源于“搜狗实验室数据库”的“全网新闻数据”http://www.sogou.com/labs/resource/cs.php
* 该数据是直接爬取的网页xml格式信息，存在大量的格式标签文件(见下图)
* 因此，第一步就是提取xml格式中‘<content>’标签内的有效数据

#### 2、xml格式有效信息提取
* xml格式信息的<docs></docs>标签缺失，首先需要补齐
* 利用urllib工具进行xml文件解析，获取url（域名）和content（正文）标签信息
* url中的"&"要用"&amp;"替换才能解析url

In [1]:
import os,random
from xml.dom import minidom
from urllib.parse import urlparse
import codecs

In [2]:
# 预处理xml文件，解决docs,url,content标签问题
def file_fill(file_dir,half_dir):
    # 查看half_dir文件夹下的文件夹和文件目录
    for _,_,files in os.walk(file_dir):
        # 判断是否存在half_dir，如果没有则创建
        if not os.path.exists(half_dir):
            os.makedirs(half_dir)
        for f in files:
            tmp_dir = half_dir+'/'+f
            text_init_dir = file_dir+'/'+f
            # 遍历文件夹下的每一篇xml文件
            with open(text_init_dir, 'r', encoding='gb18030') as source_file:
                start,end = '<docs>\n','</docs>'
                line_content = source_file.readlines()
                # 在目标文件夹中创建新文件保存预处理后的文件
                with open(tmp_dir, 'w+', encoding='utf-8') as handle_file:
                    # 添加'<docs>'头标签
                    handle_file.write(start)
                    for line in line_content:
                        # 处理url中的‘&’符号
                        text = line.replace('&', '&amp;')
                        # 添加'</docs>'头标签
                        handle_file.write(text)
                    handle_file.write(end)

#### 3、定义labels和check_class分类判别函数
* labels为url与文章类别的映射字典
* check_class则直接判断url是否在分类字典中

In [3]:
# 建立url和类别的映射字典
# 经过样本分析已获知，数据主要分为以下几类
labels = {'auto':'汽车','tech':'科技',
          'health':'健康','sports':'体育',
          'house':'房产','edu':'教育',
          'lady':'女人','eladies':'女人',
          'mil':'军事','military':'军事',
          'money':'财经','finance':'财经',
          'cul':'文化','culture':'文化',
          '2008':'奥运'
         }

In [4]:
# 检查url对应的文章是否在分类字典中
def check_class(url_lb,labels):
    if url_lb in labels:
        return True
    return False

#### 4、运行程序，获取预处理后的数据
* 获取预处理后的xml文件

In [6]:
file_dir = './SogouCA.reduced'
half_dir = './sougou_half'
file_fill(file_dir,half_dir)

#### 5、获取部分样本文件
* 由于数据量太大，进行简化处理，
* 从half_dir中随机选取n个文件存入after_dir文件夹中

In [7]:
# 部分样本获取函数
def choice_files(half_dir,choice_dir,n):
    if not os.path.exists(choice_dir):
        os.makedirs(choice_dir)
    for _,_,files in os.walk(half_dir):
        file_list = random.sample(files,n)
        for file in file_list:
            with open(half_dir+'./'+file,'r',encoding='utf-8') as f1:
                doc = f1.read()
                path = './'+choice_dir+'./'+file
                with open(path,'a+',encoding='utf-8') as f2:
                    f2.write(doc)
    print('文件存储完毕')
    return file_list

#### 6、提取训练集所需的数据
* 随机抽取10个文件作为训练源数据
* 抽取2个不同于训练集的数据作为测试源数据存放入test_choice中

In [8]:
choice_dir = './train_choice'
file_list = choice_files(half_dir,choice_dir,10)
print(file_list)

文件存储完毕
['news.allsites.380806.txt', 'news.allsites.680806.txt', 'news.allsites.1510806.txt', 'news.allsites.630806.txt', 'news.allsites.110806.txt', 'news.allsites.330806.txt', 'news.allsites.510806.txt', 'news.allsites.1120806.txt', 'news.allsites.080806.txt', 'news.allsites.180806.txt']


#### 7、提取文档文本内容
* 遍历预处理好的xml文件，提取content标签中的文本并重新保存

In [9]:
def file_read(file_dir,labels,path):
    # 遍历获取file_dir文件夹下的文件夹和文件目录
    for _,_,files in os.walk(file_dir):
        for f in files:
            filename = file_dir+'/'+f
            # js标签处理
            doc = minidom.parse(filename)
            root = doc.documentElement
            claimtext = root.getElementsByTagName('content')
            claimurl = root.getElementsByTagName('url')
            for ind in range(len(claimurl)):
                if claimtext[ind].firstChild == None:
                    continue
                # 获取url对象
                url = urlparse(claimurl[ind].firstChild.data)
                # 获取url中体现文章类型的关键字符串
                url_lb = url.hostname.strip().split('.')[0]
                # 判断url_lb对应的文章类型
                if check_class(url_lb,labels):
                    # 自动创建文件夹path（存放所有本阶段处理后的文件）
                    if not os.path.exists(path):
                        os.makedirs(path)
                    # 在path路径下自动创建文章类别文件夹
                    if not os.path.exists(path+'./'+labels[url_lb]):
                        os.makedirs(path+'./'+labels[url_lb])
                    file_name = path+'./'+labels[url_lb]+'./'+"{}.txt"\
                    .format(labels[url_lb])
                    # 在每一个分类文件夹下创建处理后的文本文件
                    with open(file_name,"a+",encoding='utf-8') as file_in:
                        file_in.write(claimtext[ind].firstChild.data+'\n')

In [10]:
# 定义存放最后文件的文件夹地址
path = "./train_after"
# 将选好的文件进行纯文本提取和分类存储
file_dir = 'train_choice'
file_read(file_dir,labels,path)

In [11]:
# 将选好的文件进行纯文本提取和分类存储
file_dir = 'test_choice'
# 定义存放最后文件的文件夹地址
test_path = "./test_after"

file_read(file_dir,labels,test_path)

#### 8、预处理数据小结
* 到该阶段已基本获取所需要的纯文本文件，走完了万里长征第一步
* 文本预处理阶段主要是细节问题比较多包括以下方面：
* 1、xml标签的处理
* 2、文本的编码格式要注意
* 3、os.walk和os.makedirs的使用


### 三、数据清洗
* 在数据预处理基础之上进一步对数据格式进行清洗
* 包括删除特殊字符，去除标点和中文分词等工作
* 需要自定义函数，并使用相关工具包如jieba等

#### 1、提取文本中的汉字数据
* 此处可以直接通过提取文章中的汉字方式过滤数字和特殊符号
* 去除中文停用词
###### 有些项目可能会专门去除数据中的url及无效数字和标点

In [12]:
# 导入字符串匹配需要的工具包
import re
import jieba

In [13]:
# 获取停用词列表
with open('files/stopwords_1208.txt','r',
          encoding='utf-8-sig') as f:
   stopwords = f.readlines()

In [14]:
# # 识别是否存在数字或字符
# def hasNumChar(strs):
#     return bool(re.search(r'\d', strs))

# # 识别是否是字母
# def isChar(strs):
#     return bool(re.search(r'[a-zA-Z]', strs))

# # 识别是否含有特殊字符和标点
# def hasSymbol(strs):
#     pat = r'''[\s+\.\!\/_,$%^*(+\"\']+|[+——！，［“ ”。］：‘ ’？、~@#￥%……&*（）．]'''
#     return bool(re.search(pat, strs))

# 判断词语是否为汉字
def isHans(strs):
    pat = r'[\u4e00-\u9fa5]+'
    return bool(re.match(pat, strs))

# 判断分词的基本特征
def check(word):
    word = word.strip().replace(' ','')
    if isHans(word):
        return True
    else:
        return False

# 综合处理
def preprocessing(sens):
    res = []
    for word in sens:
        if check(word):
            res.append(word)
    return res

#### 2、获取训练集及测试集的分类数据
* 为使项目结构更加清晰可以建立多层文件夹路径
* 主要利用os.walk和os.makedirs等方法

In [15]:
# 读取处理文件数据
def handle_files(file_path,new_path):
    # 创建new_path存储处理好后的文件
    if not os.path.exists(new_path):
        os.makedirs(new_path)
    # 进入预处理后的主文件夹sougou_after
    for _,dirs,_ in os.walk(file_path):
        print(dirs)
        for d in dirs:
            d_path = file_path+'./'+d
            new_d_path = new_path+'./'+d
            # 在new_path文件夹中穿件各种分类文件夹
            if not os.path.exists(new_d_path):
                os.makedirs(new_d_path)
            # 进入各种分类文件夹
            for _,_,files in os.walk(d_path):
                for file in files:
                    d_p_file = d_path+'./'+file
                    n_d_file = new_d_path+'./'+file
#                     print('------------')
#                     print(d_p_file)
                    # 打开分类文件夹中的文件
                    with open(d_p_file,'r',encoding='utf-8') as f1:
                        docs = f1.readlines()
#                         print('文件读取完毕')
                        # 创建一个空列表收集处理后的句子
                        res_docs = []
                        # 读取每一行的数据
                        for doc in docs:
                            # 将每行的数据进行分词
                            sens = jieba.cut(doc)
                            # 提取文本中的汉语数据
                            res = preprocessing(sens)
                            result = ' '.join(res)
                            result += '\n'
                            res_docs.append(result)
                        print(len(res_docs))
                        # 将处理好后的res_docs里的句子写入新的文件夹
                        with open(n_d_file,'a+',
                                  encoding='utf-8') as f2:
                            for docum in res_docs:
                                f2.write(docum)
                            print('文件写入小结')
    print('文章处理完毕')                      

In [16]:
# 训练文本数据格式化
# test_path = './sougoo'
train_new_path = 'train_final'
handle_files(path,train_new_path)

Building prefix dict from the default dictionary ...


['体育', '健康', '军事', '奥运', '女人', '房产', '教育', '文化', '汽车', '科技', '财经']


Dumping model to file cache C:\Users\Python\AppData\Local\Temp\jieba.cache
Loading model cost 1.302 seconds.
Prefix dict has been built succesfully.


9699
文件写入小结
359
文件写入小结
670
文件写入小结
2772
文件写入小结
3342
文件写入小结
558
文件写入小结
2585
文件写入小结
369
文件写入小结
2833
文件写入小结
811
文件写入小结
12001
文件写入小结
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
文章处理完毕


In [17]:
# 测试文本数据格式化
test_new_path = 'test_final'
handle_files(test_path,test_new_path)

['体育', '健康', '军事', '奥运', '女人', '房产', '教育', '文化', '汽车', '科技', '财经']
965
文件写入小结
42
文件写入小结
65
文件写入小结
274
文件写入小结
322
文件写入小结
73
文件写入小结
235
文件写入小结
38
文件写入小结
278
文件写入小结
87
文件写入小结
1193
文件写入小结
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
文章处理完毕


####  3、列表化训练集和测试集数据

In [18]:
# 定义测试数据矩阵函数
def label_cont2list(data_path):
    # 定义根目录内容和标签列表
    dir_cont_list,dir_label_list = [],[]
    for _,dirs,_ in os.walk(data_path):
        for d in dirs:
            # 进入分类的子文件夹
            d_dir = data_path+'/'+d
            for _,_,files in os.walk(d_dir):
                # 定义子文件夹内容列表
                d_cont_list = []
                for file in files:
                    file_name = d_dir+'/'+file
                    # 读取分类子文件夹内的文件
                    with open(file_name,encoding='utf-8') as f:
                        # 将文件内的每行数据作为元素存入列表
                        d_c_list = [k.strip() for k in f.readlines()]
                    d_cont_list.extend(d_c_list)
            # 分类文件夹级别内容列表合并
            dir_cont_list.extend(d_cont_list)
            # 将每个子文件夹的所有标签列表元素进行统一化后合并
            dir_label_list.extend(
                [str(d) for _ in range(len(d_cont_list))])
    return dir_label_list,dir_cont_list

In [19]:
# 获取训练数据集的标签列表和内容列表
train_path = 'train_final'
train_label_list,train_cont_list = label_cont2list(train_path)

In [20]:
# 获取测试集的标签列表和内容列表
test_path = 'test_final'
test_label_list,test_cont_list = label_cont2list(test_path)

In [21]:
print(len(train_cont_list))
# print(dir_cont_list)
print(train_cont_list[:10])

35999
['期 的 中奖号码 是 从 大小 上 分析 本期 开出 组合 本期 小数 仍 占优势 但 本期 基本 恢复 平稳 预计 下期 会 调整 平稳 出现 提示 下期 关注 或 从 奇偶 上 分析 本期 奇偶比 为 本期 奇数 略有 优势 本期 奇偶 数 做出 调整 估计 下期 可能 会 平稳 出现 提示 下期 关注 或 的 组合 从和值 上 分析 本期 和 值 开出 本期 和 值 点位 较 上期 有所 回升 近期 和 值 点位 波动 较大 预计 下期 可能 还会 出现 小幅 上升 估计 会 在 适中 点位 出现 下期 关注 之间 从 遗漏 总值 上 分析 本期 遗漏 总值 为 本期 遗漏 总值 仍 较 低 开出 号码 多以 重码 开出 预计 下期 可能 会 平稳 出现 下期 关注 之间', '新浪 体育讯 北京 时间 月 日 凌晨 国际田联 大奖赛 捷克 奥斯特 拉瓦 站 的 比赛 陆续 展开 在 男子 米 的 比赛 中 刚刚 打破 了 男子 百米 世界纪录 的 牙买加 小将 博 尔特 以 秒 的 成绩 创造 了 米 项目 的 赛季 最好 成绩 网页 不 支持', '您 所在 的 位置 腾讯 首页 体育频道 图片 滚动 图集 正文 第 页 图 腾讯 体育讯 北京 时间 月 日 英国女王 草地 杯 第二轮 比赛 全面 展开 赛会 头号 种子 纳达尔 以 和 击败 持 外卡 参赛 的 瑞典 老将 比约克 曼 轻松 斩获 本年度 草地 赛季 的 首场 胜利', '第 页 图 得分 后卫 杰里 韦斯特 他 的 绰号 叫作 关键球 先生 他 的 能力 无需 用 语言 来 描述 只 需要 注意 这样 一个 事实 在 这支 湖人队 史 最强 阵容 里 科比 不得不 被 他 挤 到 了 小 前锋 位置 上 分类 信息 企业 服务 招商 信息 热点 信息 热门 推荐 体育', '英超 新 赛季 赛程 已经 公布 曼联 首轮 将 主场 出战 纽卡 切尔西 主场 战 朴茨茅斯 阿森纳 主场 对 西布朗 利物浦 则 需 作客 桑德兰 网易 体育 月 日 消息 北京 时间 周一 傍晚 点 赛季 英超 赛程 正式 对外 公布 英超 新 赛季 将 从 月 日 的 社区 盾 开始 对阵 双方 是 上赛季 英超 冠军 曼联 和 足总杯 冠军 朴茨茅斯 随后 一周 在 月 日 英超 首轮 将 

#### 4、生成词汇表
* 收集训练集文本内容列表词作为词汇表

In [23]:
# 导入词汇收集工具包
from collections import Counter

# 定义获取词汇表的函数
def getVocabList(content_list, vocab_size):
    str_allContent = ''.join(content_list)
    counter = Counter(str_allContent)
    vocab_list =[k[0] for k in counter.most_common(vocab_size)]
    return vocab_list

# 生成词汇表文件
def generateVocabList(content_list, vocab_size):
    vocab_list = getVocabList(content_list, vocab_size)
    print(vocab_list[:10])
    with open('files/vocab_list.txt', 'w+', encoding='utf-8') as f:
        for vocab in vocab_list:
            f.write(vocab+'\n')
        print('词汇表创建完成')

In [24]:
generateVocabList(train_cont_list, 5000)

[' ', '的', '一', '在', '是', '中', '国', '有', '不', '了']
词汇表创建完成


#### 5、词汇表词向量化
* 利用词汇表文件将数据进行词向量化

In [25]:
# 读取词汇表文件数据，存入列表中
with open('files/vocab_list.txt',encoding='utf-8') as f:
    vocab_list = [k.strip() for k in f.readlines()]

# 将词汇列表中的数据转化为键为数据，值为序号的字词汇典
word2id_dict = dict([(b,a) for a,b in enumerate(vocab_list)])

In [26]:
# 定义词向量化函数，将数据转化为词向量
def list2vector(word2id_dict,train_cont_list):
    # 定义一个词汇字典与文本内容的映射关系
    cont2id_list = lambda content:[
        word2id_dict[
            word] for word in content if word in word2id_dict]
    # 将训练集数据映射为词汇字典对应的词向量
    train_idlist_list = [cont2id_list(
        content) for content in train_cont_list]
    return train_idlist_list

### 四、模型的建立
* 建立CNN模型进行数据训练和预测

#### 1、导入数据包

In [87]:
# 导入相关数据包
import tensorflow as tf 
import tensorflow.contrib.keras as kr 
from sklearn.preprocessing import LabelEncoder as spl

#### 2、固定参数初始化

In [88]:
# 固定参数初始化
vocab_size = 5000 # 词汇表的大小
seq_length = 600 # 序列长度
embedding_dim = 64 # 词向量维度
num_classes = 11 # 文章的类别
num_filters = 256 # 卷积核数目
kernel_size = 5 # 卷积核尺度
hidden_dim = 128 # 全连接层神经元数量
drop_keep_prob = 0.5 # dropout保留比例
learning_rate = 0.001 # 学习率
batch_size = 64 # 每批次训练样本大小

#### 3、样本集合格式化

In [105]:
# 获取训练样本映射的词向量矩阵
train_idlist_list = list2vector(
    word2id_dict,train_cont_list)

In [90]:
print(train_idlist_list[:10])

[[62, 1, 5, 601, 373, 742, 4, 151, 11, 134, 10, 38, 592, 46, 62, 77, 27, 254, 106, 46, 62, 134, 116, 502, 556, 444, 239, 118, 46, 62, 91, 46, 1018, 446, 162, 551, 234, 148, 40, 62, 16, 199, 242, 162, 551, 27, 56, 167, 190, 40, 62, 102, 308, 355, 151, 579, 1317, 10, 38, 592, 46, 62, 579, 1317, 55, 13, 46, 62, 579, 116, 578, 7, 444, 239, 46, 62, 579, 1317, 116, 350, 27, 199, 242, 633, 148, 40, 62, 58, 36, 16, 162, 551, 27, 56, 167, 190, 40, 62, 102, 308, 355, 1, 254, 106, 151, 20, 245, 10, 38, 592, 46, 62, 20, 245, 77, 27, 46, 62, 20, 245, 66, 124, 251, 10, 62, 7, 109, 240, 301, 250, 62, 20, 245, 66, 124, 506, 72, 251, 11, 234, 148, 40, 62, 58, 36, 122, 16, 27, 56, 134, 302, 10, 301, 633, 148, 16, 3, 647, 5, 66, 124, 27, 56, 40, 62, 102, 308, 110, 126, 151, 1130, 1459, 192, 245, 10, 38, 592, 46, 62, 1130, 1459, 192, 245, 13, 46, 62, 1130, 1459, 192, 245, 502, 251, 261, 77, 27, 373, 742, 64, 28, 99, 742, 77, 27, 234, 148, 40, 62, 58, 36, 16, 162, 551, 27, 56, 40, 62, 102, 308, 110, 126], 

In [106]:
# 将训练样本的文本矩阵转化为等长度的ndarray类型
train_X = kr.preprocessing.sequence.pad_sequences(
    train_idlist_list, seq_length)

In [107]:
# 将训练样本的标签矩阵进行格式化
labelEncoder = spl()
train_y = labelEncoder.fit_transform(train_label_list)
# 将训练集标签矩阵转化为独热编码格式
train_Y = kr.utils.to_categorical(train_y, num_classes)

In [126]:
# 清除默认图形堆栈并重置全局默认图形
tf.reset_default_graph()
with tf.name_scope('input_layer'):
    # 利用占位符创建索引概率分布矩阵
    X_holder = tf.placeholder(tf.int32, [None, seq_length])
    Y_holder = tf.placeholder(tf.float32, [None, num_classes])

In [109]:
# 获取测试样本映射的词向量矩阵
test_idlist_list = list2vector(
    word2id_dict,test_cont_list)
# 将测试样本的文本矩阵转化为等长度的ndarray类型
test_X = kr.preprocessing.sequence.pad_sequences(
    test_idlist_list,seq_length)
# 将测试样本的标签矩阵进行格式化
test_y = labelEncoder.fit_transform(test_label_list)
# 将测试集标签矩阵转化为独热编码格式
test_Y = kr.utils.to_categorical(test_y, num_classes)

#### 4、CNN模型训练流程

In [120]:
# CNN前向传播流程
# 创建词向量的变量
embedding = tf.get_variable('embedding',[vocab_size, embedding_dim])
# 选取词向量索引矩阵对应的元素,作为输入层数据
embedding_inputs = tf.nn.embedding_lookup(embedding,X_holder)
with tf.name_scope('conv'):
    # 创建一维卷积函数
    conv = tf.layers.conv1d(embedding_inputs, num_filters, kernel_size)
    # 利用横向局部最大值进行池化
    max_pooling = tf.reduce_max(conv, reduction_indices=[1])
with tf.name_scope('full_connect'):
    # 创建全连接层
    full_connect = tf.layers.dense(max_pooling, hidden_dim)
    # 降采样，防止神经网络过拟合，保留75%神经元参与训练
    full_connect_drop = tf.contrib.layers.dropout(full_connect,keep_prob=0.75)
    # 利用relu函数进行全连接层激活处理
    full_connect_activate = tf.nn.relu(full_connect_drop)
with tf.name_scope('predict'):
    # 将输出层前全连接层的输出进行训练，得出预测得分
    softmax_pre = tf.layers.dense(full_connect_activate, num_classes)
    # 将预测得分进行softmax激活处理，得到各种类别的预测概率
    pred_Y = tf.nn.softmax(softmax_pre)

# CNN后向传播流程
with tf.name_scope('loss'):
    # 计算输出标签与真实标签的交叉熵
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(
        labels=Y_holder, logits=softmax_pre)
    # 计算损失(交叉熵均值)误差均值
    loss = tf.reduce_mean(cross_entropy)
    tf.summary.scalar('loss',loss)
with tf.name_scope('optimizer'):
    # 利用Adam算法创建自动优化器,学习率为0.001
    optimizer = tf.train.AdamOptimizer(learning_rate)
    # 对损失函数进行全局优化
    train = optimizer.minimize(loss)
merged = tf.summary.merge_all()
# 获取真实标签与预测标签的元素布尔型矩阵
isCorrect = tf.equal(tf.argmax(Y_holder, 1), tf.argmax(pred_Y, 1))
# 将布尔型矩阵转化为0-1矩阵，并浮点化，并求预测准确率
accuracy = tf.reduce_mean(tf.cast(isCorrect, tf.float32))

In [84]:
# CNN前向传播流程
# 创建词向量的变量
embedding = tf.get_variable('embedding',[vocab_size, embedding_dim])
# 选取词向量索引矩阵对应的元素,作为输入层数据
embedding_inputs = tf.nn.embedding_lookup(embedding,X_holder)
# 创建一维卷积函数
conv = tf.layers.conv1d(embedding_inputs, num_filters, kernel_size)
# 利用横向局部最大值进行池化
max_pooling = tf.reduce_max(conv, reduction_indices=[1])
# 创建全连接层
full_connect = tf.layers.dense(max_pooling, hidden_dim)
# 降采样，防止神经网络过拟合，保留75%神经元参与训练
full_connect_drop = tf.contrib.layers.dropout(full_connect,keep_prob=0.75)
# 利用relu函数进行全连接层激活处理
full_connect_activate = tf.nn.relu(full_connect_drop)
# 将输出层前全连接层的输出进行训练，得出预测得分
softmax_pre = tf.layers.dense(full_connect_activate, num_classes)
# 将预测得分进行softmax激活处理，得到各种类别的预测概率
pred_Y = tf.nn.softmax(softmax_pre)

# CNN后向传播流程
# 计算输出标签与真实标签的交叉熵
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(
    labels=Y_holder, logits=softmax_pre)
# 计算损失(交叉熵均值)误差均值
loss = tf.reduce_mean(cross_entropy)
# 利用Adam算法创建自动优化器,学习率为0.001
optimizer = tf.train.AdamOptimizer(learning_rate)
# 对损失函数进行全局优化
train = optimizer.minimize(loss)
# 获取真实标签与预测标签的元素布尔型矩阵
isCorrect = tf.equal(tf.argmax(Y_holder, 1), tf.argmax(pred_Y, 1))
# 将布尔型矩阵转化为0-1矩阵，并浮点化，并求预测准确率
accuracy = tf.reduce_mean(tf.cast(isCorrect, tf.float32))

ValueError: Variable embedding already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Users\Python\venvenv\lib\site-packages\tensorflow\python\framework\ops.py", line 1770, in __init__
    self._traceback = tf_stack.extract_stack()
  File "C:\Users\Python\venvenv\lib\site-packages\tensorflow\python\framework\ops.py", line 3274, in create_op
    op_def=op_def)
  File "C:\Users\Python\venvenv\lib\site-packages\tensorflow\python\util\deprecation.py", line 488, in new_func
    return func(*args, **kwargs)


### 五、模型的训练与预测
* 进行2000轮训练
* 观察损失函数loss和预测准确率accuracy

#### 1、创建模型保存路径

In [121]:
# 创建保存对象
saver = tf.train.Saver()
# 最优结果
best_acc = 0
# 创建保存训练模型的路径
model_path = 'save_models/'
if not os.path.exists(model_path):
    os.makedirs(model_path)
save_path = os.path.join(path,'best')

#### 2、会话初始化

In [122]:
# 添加节点用于初始化全部变量
init = tf.global_variables_initializer()
# 创建会话
sess = tf.Session()
sess.run(init)

#### 3、创建数据流图可视化实例
* 利用tensorboard进行数据流图可视化
* 查看命令：（进入主程序文件所在路径地址执行）tensorboard --logdir ./ --host localhost
* 在浏览器中输入：localhost:6006

In [123]:
# 创建FileWriter实例，并传入当前会话加载的数据流图
writer = tf.summary.FileWriter('./summary/linear-regression-1', sess.graph)

#### 4、训练代码执行

In [125]:
for i in range(20):
    # 随机选取64个样本的索引
    sel_index = random.sample(
        list(range(len(train_y))), k=batch_size)
    # 批量选择训练样本
    batch_X = train_X[sel_index]
    batch_Y = train_Y[sel_index]
    # 运行训练模型
    sess.run(train, feed_dict={
        X_holder:batch_X, Y_holder:batch_Y})
    rs=sess.run(merged)
    writer.add_summary(rs, i)
    # 抽取样本观察训练效果
    i += 1
    if i % 2 == 0:
        # 随机选择200个样本进行训练
        sel_index = random.sample(
            list(range(len(train_y))), k=200)
        batch_X = train_X[sel_index]
        batch_Y = train_Y[sel_index]
        # 获取损失值loss_val和准确度accuracy_val
        loss_val,accuracy_val = sess.run(
            [loss,accuracy], feed_dict={
                X_holder:batch_X, Y_holder:batch_Y})
        # 将最优的模型保存下来
        if accuracy_val > best_acc:
            best_acc = accuracy_val
            saver.save(sess=sess, save_path=save_path)
        # 打印损失值和准确度
        print('第{}轮训练，loss值:{:.4f}，accuracy值：{:.4f}'.format(
            i,loss_val,accuracy_val))
writer.close()

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_1' with dtype float and shape [?,11]
	 [[node Placeholder_1 (defined at <ipython-input-119-8a7ab58dd663>:5)  = Placeholder[dtype=DT_FLOAT, shape=[?,11], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder_1', defined at:
  File "C:\ProgramData\Anaconda3\Lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\Lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 478, in start
    self.io_loop.start()
  File "C:\ProgramData\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 1

### 六、混淆矩阵
* 通过混淆矩阵评估模型的参数

#### 1、导入工具包

In [38]:
# 导入相关工具包
import numpy as np 
import pandas as pd 
from sklearn.metrics import confusion_matrix 

In [39]:
print(len(test_X))

3572


#### 2、数据格式化

In [40]:
# 定义批量预测函数，将结果转化为ndarray类型
def predictAll(test_X, batch_size=100):
    pred_val_list = []
    for i in range(0,len(test_X),batch_size):
        select_X = test_X[i:i+batch_size]
        pred_val = sess.run(pred_Y, {X_holder:select_X})
        pred_val_list.extend(pred_val)
    return np.array(pred_val_list)

#### 3、获取混淆矩阵

In [41]:
# 运行模型进行预测
Y = predictAll(test_X)
# 获取局部最大可能预测值的位置矩阵
y = np.argmax(Y, axis=1)
# 格式化预测标签列表
pred_label_list = labelEncoder.inverse_transform(y)
# 获取混淆矩阵
content_matrix = confusion_matrix(test_label_list,pred_label_list)
pd.DataFrame(content_matrix,columns=labelEncoder.classes_,
            index=labelEncoder.classes_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


,体育,健康,军事,奥运,女人,房产,教育,文化,汽车,科技,财经
体育,921,0,0,37,3,0,0,0,2,0,2
健康,0,27,0,0,13,0,1,1,0,0,0
军事,1,0,56,2,1,0,0,1,0,0,4
奥运,41,0,0,214,3,0,4,0,1,0,11
女人,2,0,1,0,315,0,1,1,0,0,2
房产,1,0,0,1,0,51,1,1,0,1,17
教育,1,0,0,1,3,0,229,0,0,0,1
文化,2,0,5,3,8,0,0,18,1,0,1
汽车,2,0,1,2,1,0,1,0,266,0,5
科技,3,0,0,0,2,1,3,0,1,58,19


#### 4、预测分析小结
* 绝大多数样本预测都比较准确。
* 奥运和体育出现交叉，说明二者有较大的关联性，这与事实相符。

### 七、报告分析

#### 1、定义并执行评估数据

In [69]:
from sklearn.metrics import precision_recall_fscore_support as prf

# 定义模型评估函数
def eval_model(true_y,pred_y,labels):
    # 计算每个类别的测准率，召回率，f1得分
    pred,recall,f1,s = prf(true_y, pred_y)
    # 计算上面各参数的总体值
    tt_pred = np.average(pred, weights=s)
    tt_recall = np.average(recall, weights=s)
    tt_f1 = np.average(f1, weights=s)
    tt_s = np.sum(s)
    res1 = pd.DataFrame({
        'Label':labels,
        'Prediction':pred,
        'Recall':recall,
        'F1':f1,
        'Support':s
    })
    res2 = pd.DataFrame({
        'Label':['总体'],
        'Prediction':tt_pred,
        'Recall':tt_recall,
        'F1':tt_f1,
        'Support':tt_s        
    })
    res2.index = [999]
    res = pd.concat([res1, res2])[['Label','Prediction','Recall','F1','Support']]
    return res,res1,res2

In [70]:
res,res1,res2 = eval_model(test_label_list, 
           pred_label_list, labelEncoder.classes_)

#### 2、可视化评估数据

In [72]:
print(res)

    Label  Prediction    Recall        F1  Support
0      体育    0.943648  0.954404  0.948995      965
1      健康    1.000000  0.642857  0.782609       42
2      军事    0.848485  0.861538  0.854962       65
3      奥运    0.795539  0.781022  0.788214      274
4      女人    0.882353  0.978261  0.927835      322
5      房产    0.910714  0.698630  0.790698       73
6      教育    0.942387  0.974468  0.958159      235
7      文化    0.818182  0.473684  0.600000       38
8      汽车    0.974359  0.956835  0.965517      278
9      科技    0.892308  0.666667  0.763158       87
10     财经    0.949097  0.968986  0.958938     1193
999    总体    0.926561  0.926932  0.924823     3572


### 八、项目总结

* 本项目以“搜狗实验室数据”的真实新闻数据为数据来源，预测结果具有客观性。
* 本项目全流程展示了数据预处理、数据清洗、模型的建立及评估的基本思路和实现代码。
* 本项目利用CNN算法建立模型，经过2000轮训练获得的准确率高达99%，可以投入实际应用。